### Connect to AWS MySQL database using credentials in config.py, create initial database, then close connection

In [ ]:
import mysql.connector
from mysql.connector import errorcode
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd
)

print(cnx) # it will print a connection object if everything is fine

In [ ]:
cursor = cnx.cursor()

In [ ]:
db_name = 'movies'

In [ ]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [ ]:
cnx.close()

### Create Table for Actors

In [ ]:
db_name = 'movies'

In [ ]:
import mysql.connector
from mysql.connector import errorcode
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)

print(cnx) # it will print a connection object if everything is fine

In [ ]:
cursor = cnx.cursor()

In [ ]:
create_actors_query = """
CREATE TABLE actors (
actor_id INT NOT NULL,
name VARCHAR(255),
is_vegan BOOLEAN,
gender VARCHAR(255),
dob DATETIME,
height INT,
PRIMARY KEY(actor_id)
);
"""

In [ ]:
cursor.execute(create_actors_query)

In [ ]:
cnx.commit()

In [ ]:
cnx.close()

### Create Table for Movies

In [ ]:
db_name = 'movies'

In [ ]:
import mysql.connector
from mysql.connector import errorcode
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)

print(cnx) # it will print a connection object if everything is fine

In [ ]:
cnx.close()

In [ ]:
cursor = cnx.cursor()

In [ ]:
create_movies_query = """
CREATE TABLE movies (
movie_id INT NOT NULL,
imdb_id VARCHAR(255),
title VARCHAR(255),
release_date DATETIME,
vote_avg FLOAT,
vote_count INT,
mpaa_rating VARCHAR(255),
runtime INT,
genres VARCHAR(255),
dom_gross INT,
PRIMARY KEY(movie_id)
);
"""

In [ ]:
cursor.execute(create_movies_query)

In [ ]:
cnx.commit()

In [ ]:
cnx.close()

### Create cross-reference table connecting actors and movies

In [ ]:
db_name = 'movies'

In [ ]:
import mysql.connector
from mysql.connector import errorcode
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)

print(cnx) # it will print a connection object if everything is fine

In [ ]:
cursor = cnx.cursor()

In [ ]:
create_actor_movie_xr_query = """
CREATE TABLE casts (
actor_id INT NOT NULL,
movie_id INT NOT NULL,
order_in_cast INT,
FOREIGN KEY(actor_id) REFERENCES actors(actor_id),
FOREIGN KEY(movie_id) REFERENCES movies(movie_id)
);
"""

In [ ]:
cursor.execute(create_actor_movie_xr_query)

In [ ]:
cnx.commit()

In [ ]:
cnx.close()

### Scrape Movies from The Movie DB

In [104]:
import json
import requests
import config

In [103]:
params = {'api_key' : config.movie_db_key}

In [ ]:
config.movie_db_key

In [ ]:
headers

In [102]:
def get_movie_details(movie_id):
    response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}", params=params)
    if response.ok:
        return json.loads(response.content)
    else:
        return None

In [ ]:
movie1_details = get_movie_details(14844)

In [ ]:
movie1_details

In [101]:
def parse_genres(genre_data):
    genre_names = []
    for item in genre_data:
        if 'name' in item.keys():
            genre_names.append(item['name'])
    return ', '.join(genre_names)

In [100]:
from datetime import datetime as datetime
 
def parse_movie_details(movie):
    movie_data = {}
    movie_data['movie_id'] = movie['id']
    movie_data['imdb_id'] = movie['imdb_id'] if 'imdb_id' in movie.keys() else None
    movie_data['title'] = movie['title'] if 'title' in movie.keys() else None
    movie_data['release_date'] = datetime.strptime(movie['release_date'], '%Y-%m-%d') if 'release_date' in movie.keys() else None
    movie_data['vote_avg'] = movie['vote_average'] if 'vote_average' in movie.keys() else None
    movie_data['vote_count'] = movie['vote_count'] if 'vote_count' in movie.keys() else None
    movie_data['mpaa_rating'] = None
    movie_data['runtime'] = movie['runtime'] if 'runtime' in movie.keys() else None
    movie_data['genres'] = parse_genres(movie['genres']) if 'genres' in movie.keys() else ''
    movie_data['dom_gross'] = movie['revenue'] if 'revenue' in movie.keys() else None
    return movie_data

In [ ]:
parsed_data = parse_movie_details(movie1_details)
tuple(parsed_data.values())

In [96]:
import mysql.connector
from mysql.connector import errorcode
import config

def insert_movie(movie_data):
    db_name = 'movies'
    cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        passwd = config.passwd,
        database = db_name)
    
    cursor = cnx.cursor()
    
    query = """INSERT INTO movies (movie_id,
                                  imdb_id,
                                  title,
                                  release_date,
                                  vote_avg,
                                  vote_count,
                                  mpaa_rating,
                                  runtime,
                                  genres,
                                  dom_gross)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    cursor.execute(query, tuple(movie_data.values()))
    cnx.commit()
    cnx.close()

In [107]:
import mysql.connector
from mysql.connector import errorcode
import config

def insert_movie2(movie_data):
    db_name = 'movies'
    cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        passwd = config.passwd,
        database = db_name)
    
    cursor = cnx.cursor()
    
    query = """INSERT INTO movies (movie_id,
                                  imdb_id,
                                  title,
                                  release_date,
                                  vote_avg,
                                  vote_count,
                                  mpaa_rating,
                                  runtime,
                                  genres,
                                  dom_gross)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    try:
        cursor.execute(query, movie_data)
    except mysql.connector.IntegrityError:
        print("Duplicate! Did not add to database.")
        cnx.close()
    cnx.commit()
    cnx.close()

In [92]:
import config

def get_movie_id(movie_title, movie_year):
    params = {'api_key' : config.movie_db_key,
             'query' : movie_title,
             'year' : movie_year}
    print(f"Searching for moviedb id for {movie_title} ({movie_year})")
    response = requests.get(f"https://api.themoviedb.org/3/search/movie", params=params)
    if response.ok:
        print("Response OK")
        try:
            content = json.loads(response.content)
            title = content['results'][0]['title']
            year = content['results'][0]['release_date'][:4]
            movie_id = content['results'][0]['id']
            print(f"Found movie {title} ({year}) with id {movie_id}")
            return movie_id
        except:
            print("Could not parse response content.")
            return None
    else:
        print("Bad Response")
        print(response.content)
        return None

In [93]:
get_movie_id('chasing liberty', 2004)

Searching for moviedb id for chasing liberty (2004)
Response OK
Found movie Chasing Liberty (2004) with id 14844


14844

In [94]:
import csv

with open('movies.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))
    
movies_list = list(map(lambda x: (x[0], x[1]), data))
movies_list

[('Chasing Liberty', '2004'),
 ("My Baby's Daddy", '2004'),
 ('Along Came Polly', '2004'),
 ("Teacher's Pet", '2004'),
 ('Torque', '2004'),
 ('The Butterfly Effect', '2004'),
 ('Win a Date with Tad Hamilton!', '2004'),
 ('The Big Bounce', '2004'),
 ('The Perfect Score', '2004'),
 ('You Got Served', '2004'),
 ('Barbershop 2: Back in Business', '2004'),
 ('Catch That Kid', '2004'),
 ('Miracle', '2004'),
 ('Pinocchio 3000', '2004'),
 ('50 First Dates', '2004'),
 ('Against the Ropes', '2004'),
 ('Confessions of a Teenage Drama Queen', '2004'),
 ('EuroTrip', '2004'),
 ('Welcome to Mooseport', '2004'),
 ('The Passion of the Christ', '2004'),
 ('Club Dread', '2004'),
 ('Twisted', '2004'),
 ('Hidalgo', '2004'),
 ('Starsky & Hutch', '2004'),
 ('Agent Cody Banks 2: Destination London', '2004'),
 ('Secret Window', '2004'),
 ('Spartan', '2004'),
 ('Dawn of the Dead', '2004'),
 ('Eternal Sunshine of the Spotless Mind', '2004'),
 ('Taking Lives', '2004'),
 ('Jersey Girl', '2004'),
 ('The Ladykillers

In [ ]:
movie_ids = []
from time import sleep

failed = []

for number, movie in enumerate(movies_list[:]):
    print(f"Trying movie number {number}")
    movie_id = get_movie_id(movie[0], movie[1])
    if movie_id:
        movie_ids.append((movie[0], movie[1], movie_id))
    else:
        failed.append(movie)
    sleep(1)

In [ ]:
# import csv

# with open('movie_ids.csv', 'w') as myfile:
#     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#     wr.writerows(movie_ids)

In [8]:
import csv

with open('movie_ids.csv', newline='') as csvfile:
    movie_id_list = list(csv.reader(csvfile))
    
movie_id_list2 = [movie_id_list[i] for i in range(len(movie_id_list)) if movie_id_list[i][2] not in [y[2] for y in movie_id_list[:i]] ]
movie_id_list = movie_id_list2

In [ ]:
movie_details = []
failed_movies = []

In [91]:
from time import sleep


def process_movies(movie_ids, details_list, failed_list):
    for number, movie in enumerate(movie_ids[:]):
        sleep(1)
        print(f"Scraping data for movie {movie[0]} ({movie[1]}) with id {movie[2]}")
        movie_details_unparsed = get_movie_details(movie[2])
        if movie_details_unparsed:
            movie_details = parse_movie_details(movie_details_unparsed)
        else:
            print(f"Unable to get movie details for movie id {movie[2]}")
            failed_list.append(movie)
            continue
        details_list.append(movie_details)

In [ ]:
process_movies(movie_id_list[:], movie_details, failed_movies)

In [2]:
import csv

with open('movie_details.csv', newline='') as csvfile:
    movie_details_pre = list(csv.reader(csvfile))
    
movie_details = list(map(lambda x: tuple(x), movie_details_pre))

In [72]:
len(movie_details)

3629

In [86]:
ids_with_details = sorted(list(map(lambda x: int(x[0]), movie_details)))

In [87]:
missing_ids = [x for x in movie_id_list if int(x[2]) not in ids_with_details]

In [98]:
movie_details2 = []
failed_movies2 = []

In [105]:
process_movies(missing_ids, movie_details2, failed_movies2)

Scraping data for movie The Butterfly Effect (2004) with id 1954
Scraping data for movie Win a Date with Tad Hamilton! (2004) with id 13476
Scraping data for movie Jersey Girl (2004) with id 9541
Scraping data for movie The Ladykillers (2004) with id 5516
Scraping data for movie Home on the Range (2004) with id 13700
Scraping data for movie The Prince and Me (2004) with id 11137
Scraping data for movie Walking Tall (2004) with id 11358
Scraping data for movie Johnson Family Vacation (2004) with id 19084
Scraping data for movie The Whole Ten Yards (2004) with id 2122
Scraping data for movie Kill Bill: Volume 2 (2004) with id 393
Scraping data for movie The Punisher (2004) with id 7220
Scraping data for movie 13 Going on 30 (2004) with id 10096
Scraping data for movie Clifford's Really Big Movie (2004) with id 19016
Scraping data for movie New York Minute (2004) with id 11025
Scraping data for movie Off the Lip (2004) with id 279709
Scraping data for movie Stateside (2004) with id 43670


Scraping data for movie Justin Bieber: Never Say Never (2011) with id 54518
Scraping data for movie The Lion of Judah (2011) with id 46256
Scraping data for movie Judy Moody and the Not Bummer Summer (2011) with id 60062
Scraping data for movie Super 8 (2011) with id 37686
Scraping data for movie Bol (2011) with id 70067
Scraping data for movie Project Nim (2011) with id 63579
Scraping data for movie Zookeeper (2011) with id 38317
Scraping data for movie Rise of the Planet of the Apes (2011) with id 61791
Scraping data for movie Glee: The 3D Concert Movie (2011) with id 67675
Scraping data for movie Abduction (2011) with id 59965
Scraping data for movie Surviving Progress (2011) with id 79172
Scraping data for movie Warrior (2011) with id 59440
Scraping data for movie The Way (2011) with id 59468
Scraping data for movie I Don't Know How She Does It (2011) with id 70868
Scraping data for movie George Harrison: Living in the Material World (2011) with id 75964
Scraping data for movie Mis

In [ ]:
# movie_details_tuples = list(map(lambda x: x.values(), movie_details))

In [ ]:
# import csv

# with open('movie_details.csv', 'w') as myfile:
#     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#     wr.writerows(movie_details_tuples)

In [4]:
from time import sleep

for movie in movie_details[838:]:
    print("INSERTING ", movie)
    insert_movie2(movie)
    sleep(0.25)

INSERTING  ('13809', 'tt1032755', 'RockNRolla', '2008-09-04 00:00:00', '6.9', '1474', '', '114', 'Action, Crime, Thriller', '25739015')
Duplicate! Did not add to database.
INSERTING  ('12113', 'tt0758774', 'Body of Lies', '2008-10-10 00:00:00', '6.6', '1574', '', '128', 'Action, Drama, Thriller', '113280098')
Duplicate! Did not add to database.
INSERTING  ('13600', 'tt0970411', 'City of Ember', '2008-10-07 00:00:00', '6.2', '1015', '', '90', 'Adventure, Family, Fantasy', '17869048')
Duplicate! Did not add to database.
INSERTING  ('14325', 'tt0469903', 'The Express', '2008-10-10 00:00:00', '7.1', '117', '', '130', 'Drama', '9793496')
INSERTING  ('10503', 'tt1045670', 'Happy-Go-Lucky', '2008-04-18 00:00:00', '6.5', '218', '', '118', 'Drama, Comedy', '18130888')
INSERTING  ('13812', 'tt1082868', 'Quarantine', '2008-10-10 00:00:00', '5.6', '618', '', '89', 'Horror, Science Fiction, Thriller', '40855419')
INSERTING  ('26772', 'tt0950749', 'Flick', '2008-10-03 00:00:00', '5.0', '3', '', '92'

INSERTING  ('10201', 'tt1068680', 'Yes Man', '2008-12-09 00:00:00', '6.5', '3674', '', '104', 'Comedy', '225990978')
INSERTING  ('10202', 'tt0960731', 'Bedtime Stories', '2008-12-24 00:00:00', '6.0', '1753', '', '99', 'Fantasy, Comedy, Family, Romance', '212874442')
INSERTING  ('4922', 'tt0421715', 'The Curious Case of Benjamin Button', '2008-12-25 00:00:00', '7.5', '7047', '', '166', 'Fantasy, Drama, Thriller, Mystery, Romance', '333932083')
INSERTING  ('16409', 'tt1046947', 'Last Chance Harvey', '2008-12-25 00:00:00', '6.2', '131', '', '92', 'Drama, Comedy, Romance', '14879423')
INSERTING  ('14306', 'tt0822832', 'Marley & Me', '2008-12-25 00:00:00', '7.2', '2855', '', '115', 'Comedy, Family', '242717113')
INSERTING  ('8285', 'tt0831887', 'The Spirit', '2008-12-25 00:00:00', '4.8', '566', '', '103', 'Action, Comedy, Thriller, Crime, Science Fiction', '39031337')
INSERTING  ('2253', 'tt0985699', 'Valkyrie', '2008-12-25 00:00:00', '6.8', '2023', '', '121', 'Drama, Thriller, History, War

INSERTING  ('16418', 'tt1398940', 'Barbie Presents: Thumbelina', '2009-02-24 00:00:00', '5.9', '236', '', '75', 'Animation, Family', '0')
INSERTING  ('28858', 'tt1239427', 'Brothers at War', '2009-03-13 00:00:00', '6.0', '2', '', '110', 'Documentary, History', '0')
INSERTING  ('16071', 'tt0418586', 'The Cake Eaters', '2007-04-29 00:00:00', '5.5', '65', '', '86', 'Drama, Romance', '0')
INSERTING  ('18405', 'tt0844708', 'The Last House on the Left', '2009-03-13 00:00:00', '6.3', '843', '', '110', 'Crime, Thriller, Horror, Drama', '32721635')
INSERTING  ('19556', 'tt1151922', 'Miss March', '2009-03-13 00:00:00', '5.3', '161', '', '89', 'Comedy, Romance', '0')
INSERTING  ('19206', 'tt0435716', 'The Perfect Sleep', '2009-03-13 00:00:00', '3.5', '10', '', '99', 'Action, Adventure, Drama, Thriller', '0')
INSERTING  ('13836', 'tt1075417', 'Race to Witch Mountain', '2009-03-12 00:00:00', '5.6', '921', '', '98', 'Adventure, Family, Fantasy, Science Fiction, Thriller, Action', '106303988')
INSERT

INSERTING  ('18065', 'tt1097013', 'Next Day Air', '2009-02-06 00:00:00', '5.2', '54', '', '84', 'Action, Comedy, Crime', '0')
INSERTING  ('30312', 'tt1049400', 'Outrage', '2009-01-01 00:00:00', '7.4', '15', '', '90', 'Documentary', '0')
INSERTING  ('17740', 'tt1032819', 'Powder Blue', '2009-05-08 00:00:00', '6.2', '107', '', '106', 'Drama', '0')
INSERTING  ('13475', 'tt0796366', 'Star Trek', '2009-05-06 00:00:00', '7.4', '6609', '', '127', 'Science Fiction, Action, Adventure', '385680446')
INSERTING  ('13448', 'tt0808151', 'Angels & Demons', '2009-05-13 00:00:00', '6.6', '4104', '', '138', 'Thriller, Mystery', '356613439')
INSERTING  ('21755', 'tt0844286', 'The Brothers Bloom', '2008-09-09 00:00:00', '6.7', '332', '', '114', 'Adventure, Comedy, Drama, Romance', '5530764')
INSERTING  ('12161', 'tt1224153', 'Jerichow', '2008-08-28 00:00:00', '6.2', '14', '', '93', 'Drama', '0')
INSERTING  ('21583', 'tt1082853', 'Management', '2009-02-08 00:00:00', '5.4', '177', '', '94', 'Comedy, Drama, 

INSERTING  ('19833', 'tt1226774', 'In the Loop', '2009-01-22 00:00:00', '7.1', '316', '', '106', 'Comedy', '6149791')
INSERTING  ('21208', 'tt1148204', 'Orphan', '2009-07-24 00:00:00', '6.8', '2656', '', '123', 'Horror, Thriller, Mystery', '41596251')
INSERTING  ('22640', 'tt1247692', 'Shrink', '2009-06-08 00:00:00', '6.2', '103', '', '104', 'Comedy, Drama', '0')
INSERTING  ('20943', 'tt1142988', 'The Ugly Truth', '2009-07-24 00:00:00', '6.5', '2104', '', '96', 'Comedy, Romance', '205298907')
INSERTING  ('22051', 'tt1185836', 'Adam', '2009-07-07 00:00:00', '6.8', '245', '', '99', 'Drama, Romance', '2277396')
INSERTING  ('20856', 'tt0775552', 'Aliens in the Attic', '2009-07-31 00:00:00', '5.5', '556', '', '86', 'Adventure, Comedy, Family, Fantasy, Science Fiction', '57881056')
INSERTING  ('21407', 'tt0844479', 'The Collector', '2009-07-09 00:00:00', '6.3', '655', '', '88', 'Horror, Thriller', '114000000')
INSERTING  ('23128', 'tt1313104', 'The Cove', '2009-07-31 00:00:00', '8.1', '347',

...

INSERTING  ('297762', 'tt0451279', 'Wonder Woman', '2017-05-30 00:00:00', '7.3', '13140', '', '141', 'Action, Adventure, Fantasy', '821847012')
Duplicate! Did not add to database.
INSERTING  ('524034', 'tt7981318', 'Cook Off', '2017-03-12 00:00:00', '0.0', '0', '', '116', 'Comedy, Romance', '0')
INSERTING  ('424510', 'tt6212496', 'Mr. Roosevelt', '2017-07-29 00:00:00', '6.0', '30', '', '90', 'Comedy', '0')
INSERTING  ('413362', 'tt6000478', 'Roman J. Israel, Esq.', '2017-11-17 00:00:00', '6.2', '536', '', '129', 'Drama', '11942780')
INSERTING  ('398818', 'tt5726616', 'Call Me by Your Name', '2017-09-01 00:00:00', '8.3', '5392', '', '132', 'Romance, Drama', '41063475')
Duplicate! Did not add to database.
INSERTING  ('353616', 'tt4765284', 'Pitch Perfect 3', '2017-12-20 00:00:00', '6.5', '2018', '', '93', 'Comedy, Music', '184724494')
INSERTING  ('399055', 'tt5580390', 'The Shape of Water', '2017-12-01 00:00:00', '7.2', '7625', '', '123', 'Drama, Fantasy, Romance', '194989655')
INSERTING

INSERTING  ('469721', 'tt3463106', 'The Cured', '2018-02-23 00:00:00', '5.5', '141', '', '95', 'Horror, Drama, Science Fiction', '0')
INSERTING  ('429467', 'tt4399952', 'The Lodgers', '2017-10-05 00:00:00', '4.8', '232', '', '93', 'Drama, Horror, Romance, Thriller', '7362')
INSERTING  ('401981', 'tt2873282', 'Red Sparrow', '2018-02-28 00:00:00', '6.5', '3627', '', '140', 'Mystery, Thriller', '147988192')
INSERTING  ('395990', 'tt1137450', 'Death Wish', '2018-03-02 00:00:00', '5.9', '1309', '', '107', 'Action, Crime, Drama, Thriller', '34017028')
INSERTING  ('353066', 'tt1291566', 'The Vanishing of Sidney Hall', '2017-10-06 00:00:00', '6.7', '105', '', '117', 'Drama, Mystery', '0')
INSERTING  ('444431', 'tt6896536', 'Foxtrot', '2017-09-02 00:00:00', '7.1', '80', '', '113', 'Drama', '0')
INSERTING  ('457312', 'tt4789822', 'Pickings', '2018-03-02 00:00:00', '6.0', '3', '', '103', 'Action, Crime, Thriller, Drama, Mystery', '0')
INSERTING  ('407451', 'tt1620680', 'A Wrinkle in Time', '2018-

INSERTING  ('385332', 'tt4463816', 'Terminal', '2018-05-11 00:00:00', '5.6', '363', '', '95', 'Thriller, Crime', '0')
INSERTING  ('467938', 'tt6738136', 'Revenge', '2018-02-07 00:00:00', '6.3', '772', '', '109', 'Action, Thriller, Horror', '692079')
INSERTING  ('383498', 'tt5463162', 'Deadpool 2', '2018-05-10 00:00:00', '7.5', '9509', '', '121', 'Action, Comedy, Adventure', '741547413')
INSERTING  ('502682', 'tt6857166', 'Book Club', '2018-05-18 00:00:00', '6.1', '355', '', '104', 'Comedy, Romance, Drama', '78720293')
INSERTING  ('458737', 'tt6053438', 'First Reformed', '2018-05-18 00:00:00', '6.9', '520', '', '113', 'Drama', '2525738')
INSERTING  ('505205', 'tt6915100', 'Pope Francis: A Man of His Word', '2018-05-18 00:00:00', '7.4', '26', '', '96', 'Documentary', '2423465')
INSERTING  ('425148', 'tt5691024', 'Show Dogs', '2018-05-10 00:00:00', '5.9', '105', '', '92', 'Adventure, Crime, Family, Comedy', '38830219')
INSERTING  ('348350', 'tt3778644', 'Solo: A Star Wars Story', '2018-05

INSERTING  ('487558', 'tt7349662', 'BlacKkKlansman', '2018-07-30 00:00:00', '7.6', '3590', '', '135', 'Crime, Drama', '48686605')
INSERTING  ('439015', 'tt5690360', 'Slender Man', '2018-08-10 00:00:00', '4.0', '826', '', '93', 'Mystery, Thriller, Horror', '11817275')
INSERTING  ('448776', 'tt4080956', 'A Prayer Before Dawn', '2018-05-24 00:00:00', '6.8', '203', '', '116', 'Drama, Action, Crime', '0')
INSERTING  ('455207', 'tt3104988', 'Crazy Rich Asians', '2018-08-15 00:00:00', '7.1', '1694', '', '121', 'Comedy, Drama, Romance', '237994561')
INSERTING  ('399360', 'tt4244998', 'Alpha', '2018-08-17 00:00:00', '6.0', '1283', '', '97', 'Adventure, Drama', '35808437')
INSERTING  ('347375', 'tt4560436', 'Mile 22', '2018-08-16 00:00:00', '6.0', '1042', '', '95', 'Action, Thriller', '0')
INSERTING  ('340613', 'tt3750872', 'The Wife', '2018-08-02 00:00:00', '7.2', '522', '', '100', 'Drama', '5634567')
INSERTING  ('385360', 'tt5179598', 'Billionaire Boys Club', '2018-07-19 00:00:00', '5.6', '155

INSERTING  ('472451', 'tt7008872', 'Boy Erased', '2018-09-24 00:00:00', '7.2', '537', '', '114', 'Drama', '7928149')
INSERTING  ('475132', 'tt2368254', 'A Private War', '2018-11-16 00:00:00', '6.9', '159', '', '106', 'Drama, War', '0')
INSERTING  ('431491', 'tt5883570', 'Bodied', '2018-11-02 00:00:00', '7.8', '46', '', '120', 'Drama, Comedy', '0')
INSERTING  ('476764', 'tt7074886', 'The Front Runner', '2018-11-06 00:00:00', '5.9', '139', '', '113', 'Drama, History', '0')
INSERTING  ('360920', 'tt2709692', 'The Grinch', '2018-11-08 00:00:00', '6.4', '1199', '', '90', 'Animation, Family, Comedy, Fantasy', '508575295')
INSERTING  ('446807', 'tt5177088', "The Girl in the Spider's Web", '2018-10-25 00:00:00', '6.0', '607', '', '117', 'Action, Crime, Thriller', '17894345')
INSERTING  ('438799', 'tt4530422', 'Overlord', '2018-11-01 00:00:00', '6.6', '1200', '', '110', 'Horror, War, Science Fiction', '41657844')
INSERTING  ('338952', 'tt4123430', 'Fantastic Beasts: The Crimes of Grindelwald', 

In [109]:
from time import sleep

for movie in movie_details2:
    print("INSERTING ", movie)
    insert_movie(movie)
    sleep(0.25)

INSERTING  {'movie_id': 1954, 'imdb_id': 'tt0289879', 'title': 'The Butterfly Effect', 'release_date': datetime.datetime(2004, 1, 22, 0, 0), 'vote_avg': 7.5, 'vote_count': 4021, 'mpaa_rating': None, 'runtime': 113, 'genres': 'Science Fiction, Thriller', 'dom_gross': 96060858}
INSERTING  {'movie_id': 13476, 'imdb_id': 'tt0335559', 'title': 'Win a Date with Tad Hamilton!', 'release_date': datetime.datetime(2004, 1, 23, 0, 0), 'vote_avg': 5.7, 'vote_count': 302, 'mpaa_rating': None, 'runtime': 95, 'genres': 'Comedy, Romance', 'dom_gross': 16980098}
INSERTING  {'movie_id': 9541, 'imdb_id': 'tt0300051', 'title': 'Jersey Girl', 'release_date': datetime.datetime(2004, 3, 25, 0, 0), 'vote_avg': 6.0, 'vote_count': 560, 'mpaa_rating': None, 'runtime': 102, 'genres': 'Comedy, Romance', 'dom_gross': 36098382}
INSERTING  {'movie_id': 5516, 'imdb_id': 'tt0335245', 'title': 'The Ladykillers', 'release_date': datetime.datetime(2004, 3, 25, 0, 0), 'vote_avg': 6.1, 'vote_count': 819, 'mpaa_rating': None

INSERTING  {'movie_id': 1970, 'imdb_id': 'tt0391198', 'title': 'The Grudge', 'release_date': datetime.datetime(2004, 10, 22, 0, 0), 'vote_avg': 5.9, 'vote_count': 1647, 'mpaa_rating': None, 'runtime': 92, 'genres': 'Horror, Mystery, Thriller', 'dom_gross': 183474602}
INSERTING  {'movie_id': 9675, 'imdb_id': 'tt0375063', 'title': 'Sideways', 'release_date': datetime.datetime(2004, 10, 22, 0, 0), 'vote_avg': 7.0, 'vote_count': 825, 'mpaa_rating': None, 'runtime': 127, 'genres': 'Comedy, Drama, Romance', 'dom_gross': 109502303}
INSERTING  {'movie_id': 866, 'imdb_id': 'tt0308644', 'title': 'Finding Neverland', 'release_date': datetime.datetime(2004, 10, 17, 0, 0), 'vote_avg': 7.3, 'vote_count': 2360, 'mpaa_rating': None, 'runtime': 106, 'genres': 'Drama', 'dom_gross': 116766556}
INSERTING  {'movie_id': 36648, 'imdb_id': 'tt0359013', 'title': 'Blade: Trinity', 'release_date': datetime.datetime(2004, 12, 8, 0, 0), 'vote_avg': 5.8, 'vote_count': 2165, 'mpaa_rating': None, 'runtime': 123, 'gen

INSERTING  {'movie_id': 13922, 'imdb_id': 'tt0326905', 'title': 'The Great Raid', 'release_date': datetime.datetime(2005, 8, 12, 0, 0), 'vote_avg': 6.7, 'vote_count': 145, 'mpaa_rating': None, 'runtime': 132, 'genres': 'Action, History, War', 'dom_gross': 10166502}
INSERTING  {'movie_id': 9913, 'imdb_id': 'tt0397101', 'title': 'The Skeleton Key', 'release_date': datetime.datetime(2005, 7, 29, 0, 0), 'vote_avg': 6.5, 'vote_count': 1076, 'mpaa_rating': None, 'runtime': 104, 'genres': 'Drama, Horror, Mystery, Thriller', 'dom_gross': 91974818}
INSERTING  {'movie_id': 17186, 'imdb_id': 'tt0403016', 'title': 'Supercross', 'release_date': datetime.datetime(2005, 8, 16, 0, 0), 'vote_avg': 5.2, 'vote_count': 53, 'mpaa_rating': None, 'runtime': 80, 'genres': 'Action, Adventure, Drama, Romance', 'dom_gross': 3102550}
INSERTING  {'movie_id': 17926, 'imdb_id': 'tt0434424', 'title': 'Undiscovered', 'release_date': datetime.datetime(2005, 8, 26, 0, 0), 'vote_avg': 4.7, 'vote_count': 20, 'mpaa_rating'

INSERTING  {'movie_id': 219, 'imdb_id': 'tt0441909', 'title': 'Volver', 'release_date': datetime.datetime(2006, 3, 16, 0, 0), 'vote_avg': 7.5, 'vote_count': 810, 'mpaa_rating': None, 'runtime': 121, 'genres': 'Comedy, Drama, Romance', 'dom_gross': 85582407}
INSERTING  {'movie_id': 1253, 'imdb_id': 'tt0443456', 'title': 'Breaking and Entering', 'release_date': datetime.datetime(2006, 12, 15, 0, 0), 'vote_avg': 6.3, 'vote_count': 115, 'mpaa_rating': None, 'runtime': 129, 'genres': 'Drama, Romance', 'dom_gross': 8974829}
INSERTING  {'movie_id': 36557, 'imdb_id': 'tt0381061', 'title': 'Casino Royale', 'release_date': datetime.datetime(2006, 11, 14, 0, 0), 'vote_avg': 7.4, 'vote_count': 6254, 'mpaa_rating': None, 'runtime': 144, 'genres': 'Adventure, Action, Thriller', 'dom_gross': 599045960}
INSERTING  {'movie_id': 14799, 'imdb_id': 'tt0470765', 'title': 'For Your Consideration', 'release_date': datetime.datetime(2006, 11, 22, 0, 0), 'vote_avg': 5.9, 'vote_count': 75, 'mpaa_rating': None, 

INSERTING  {'movie_id': 10193, 'imdb_id': 'tt0435761', 'title': 'Toy Story 3', 'release_date': datetime.datetime(2010, 6, 16, 0, 0), 'vote_avg': 7.8, 'vote_count': 9013, 'mpaa_rating': None, 'runtime': 103, 'genres': 'Animation, Family, Comedy', 'dom_gross': 1066969703}
INSERTING  {'movie_id': 81583, 'imdb_id': 'tt1337139', 'title': '45365', 'release_date': datetime.datetime(2010, 6, 18, 0, 0), 'vote_avg': 5.0, 'vote_count': 1, 'mpaa_rating': None, 'runtime': 90, 'genres': 'Documentary', 'dom_gross': 0}
INSERTING  {'movie_id': 39312, 'imdb_id': 'tt1559549', 'title': 'Restrepo', 'release_date': datetime.datetime(2010, 6, 25, 0, 0), 'vote_avg': 7.5, 'vote_count': 194, 'mpaa_rating': None, 'runtime': 93, 'genres': 'Documentary, War', 'dom_gross': 1437000}
INSERTING  {'movie_id': 47912, 'imdb_id': 'tt1280015', 'title': 'Smash His Camera', 'release_date': datetime.datetime(2010, 7, 30, 0, 0), 'vote_avg': 7.0, 'vote_count': 11, 'mpaa_rating': None, 'runtime': 87, 'genres': 'Documentary', 'do

INSERTING  {'movie_id': 87567, 'imdb_id': 'tt1931435', 'title': 'The Big Wedding', 'release_date': datetime.datetime(2013, 4, 25, 0, 0), 'vote_avg': 5.8, 'vote_count': 711, 'mpaa_rating': None, 'runtime': 90, 'genres': 'Comedy', 'dom_gross': 21819348}
INSERTING  {'movie_id': 158914, 'imdb_id': 'tt1951216', 'title': 'The Look of Love', 'release_date': datetime.datetime(2013, 4, 26, 0, 0), 'vote_avg': 5.5, 'vote_count': 57, 'mpaa_rating': None, 'runtime': 105, 'genres': 'Comedy, Drama', 'dom_gross': 0}
INSERTING  {'movie_id': 158895, 'imdb_id': 'tt1798291', 'title': 'This Is Martin Bonner', 'release_date': datetime.datetime(2013, 6, 28, 0, 0), 'vote_avg': 5.9, 'vote_count': 10, 'mpaa_rating': None, 'runtime': 83, 'genres': 'Drama', 'dom_gross': 0}
INSERTING  {'movie_id': 72190, 'imdb_id': 'tt0816711', 'title': 'World War Z', 'release_date': datetime.datetime(2013, 6, 20, 0, 0), 'vote_avg': 6.7, 'vote_count': 9487, 'mpaa_rating': None, 'runtime': 116, 'genres': 'Action, Drama, Horror, Sci

### Get Cast Info

In [20]:
import json
import requests
import config

params = {'api_key' : config.movie_db_key}

def get_movie_cast(movie_id):
    response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}/credits", params=params)
    if response.ok:
        return json.loads(response.content)
    else:
        return None

In [42]:
def parse_cast_data(data):
    parsed_data = []
    for person in data['cast']:
        person_dict = {'actor_id' : person['id'],
                       'name' : person['name'] if 'name' in person.keys() else None,
                       'is_vegan' : None,
                       'gender' : ("M" if (person['gender'] == 2) else ("F" if (person['gender'] == 1) else None)) if 'gender' in person.keys() else None, 
                       'dob' : None,
                       'height' : None,
                       'order_in_cast' : person['order'],
                       'movie_id' : data['id']
                      }
        parsed_data.append(person_dict)
    return parsed_data

### Insert Actor Data into AWS Database

In [56]:
import mysql.connector
from mysql.connector import errorcode
import config

def insert_actor(actor_data, cursor):

    
    query1 = """INSERT IGNORE INTO actors (actor_id,
                                  name,
                                  is_vegan,
                                  gender,
                                  dob,
                                  height
                                )
                VALUES (%s, %s, %s, %s, %s, %s)"""
    
    values1 = (actor_data['actor_id'],
            actor_data['name'],
            actor_data['is_vegan'],
            actor_data['gender'],
            actor_data['dob'],
            actor_data['height'])
    
    cursor.execute(query1, values1)
    
    query2 = """INSERT INTO casts (actor_id, movie_id, order_in_cast)
                VALUES (%s, %s, %s)
                """
    
    values2 = (actor_data['actor_id'],
            actor_data['movie_id'],
            actor_data['order_in_cast'])
    
    cursor.execute(query2, values2)
    

### Cast Database Population Loop

In [23]:
def populate_stuff(movie_id_list, failed_id_list):
    for number, movie in enumerate(movie_id_list):
        title = movie[0]
        year = movie[1]
        movie_id = movie[2]
        print(f"Trying movie no.{number} with id {movie[2]}: {movie[0]} ({movie[1]})")
        try:
            data = get_movie_cast(movie_id)
        except:
            print("Unable to get movie cast info.")
            failed_id_list.append(movie)
            continue
        
        try:
            parsed_data = parse_cast_data(data)
        except:
            print("Unable to parse movie cast info.")
            failed_id_list.append(movie)
            continue
            
        db_name = 'movies'
        cnx = mysql.connector.connect(
                host = config.host,
                user = config.user,
                passwd = config.passwd,
                database = db_name)
        cursor = cnx.cursor()
        
        for actor_data in parsed_data:
            try:
                insert_actor(actor_data, cursor)
                sleep(0.05)
            except:
                print(f"Failed to insert actor {actor_data['name']} (id {actor_data['actor_id']})")
                cnx.close()
                break
                      
        try:
            cnx.commit()
            cnx.close()
        except:
            failed_id_list.append(movie)
            print(f"Transaction not committed.")
                      
        sleep(0.5)
        
        

In [17]:
failed_id_list = []

In [66]:
cast_data = get_movie_cast(movie_id_list[91][2])

In [67]:
cast_data['cast']

[{'cast_id': 2,
  'character': 'Patience Phillips / Catwoman',
  'credit_id': '52fe4236c3a36847f800ca9b',
  'gender': 1,
  'id': 4587,
  'name': 'Halle Berry',
  'order': 0,
  'profile_path': '/AmCXHowNbUXpNf41dNrxNB0naM2.jpg'},
 {'cast_id': 7,
  'character': 'Tom Lone',
  'credit_id': '52fe4236c3a36847f800cab7',
  'gender': 2,
  'id': 4589,
  'name': 'Benjamin Bratt',
  'order': 1,
  'profile_path': '/5g88fU683ZXhFn0VSsBMjLHsXG1.jpg'},
 {'cast_id': 8,
  'character': 'Laurel Hedare',
  'credit_id': '52fe4236c3a36847f800cabb',
  'gender': 1,
  'id': 4430,
  'name': 'Sharon Stone',
  'order': 2,
  'profile_path': '/yICU0UWEZjWSZ9PnQRaerwn29tz.jpg'},
 {'cast_id': 9,
  'character': 'George Hedare',
  'credit_id': '52fe4236c3a36847f800cabf',
  'gender': 2,
  'id': 2192,
  'name': 'Lambert Wilson',
  'order': 3,
  'profile_path': '/jFkr2kuFfpzS1plfOlNJ6kURV6O.jpg'},
 {'cast_id': 10,
  'character': 'Ophelia',
  'credit_id': '52fe4236c3a36847f800cac3',
  'gender': 1,
  'id': 4432,
  'name': 'F

In [68]:
cast = parse_cast_data(cast_data)

In [71]:
cast[0]

{'actor_id': 4587,
 'name': 'Halle Berry',
 'is_vegan': None,
 'gender': 'F',
 'dob': None,
 'height': None,
 'order_in_cast': 0,
 'movie_id': 314}

In [49]:
db_name = 'movies'
cnx = mysql.connector.connect(
                host = config.host,
                user = config.user,
                passwd = config.passwd,
                database = db_name)
cursor = cnx.cursor()

In [57]:
insert_actor(cast[0], cursor)

In [58]:
cnx.commit()
cnx.close()

In [122]:
failed_id_list = []

In [60]:
populate_stuff(movie_id_list[:100], failed_id_list)

Trying movie no.0 with id 14844: Chasing Liberty (2004)
Trying movie no.1 with id 26710: My Baby's Daddy (2004)
Trying movie no.2 with id 5966: Along Came Polly (2004)
Trying movie no.3 with id 24034: Teacher's Pet (2004)
Trying movie no.4 with id 10718: Torque (2004)
Trying movie no.5 with id 1954: The Butterfly Effect (2004)
Failed to insert actor Ashton Kutcher (id 18976)
Trying movie no.6 with id 13476: Win a Date with Tad Hamilton! (2004)
Failed to insert actor Josh Duhamel (id 19536)
Trying movie no.7 with id 12634: The Big Bounce (2004)
Trying movie no.8 with id 13505: The Perfect Score (2004)
Trying movie no.9 with id 14114: You Got Served (2004)
Trying movie no.10 with id 21301: Barbershop 2: Back in Business (2004)
Trying movie no.11 with id 20483: Catch That Kid (2004)
Trying movie no.12 with id 14292: Miracle (2004)
Trying movie no.13 with id 23056: Pinocchio 3000 (2004)
Trying movie no.14 with id 1824: 50 First Dates (2004)
Trying movie no.15 with id 8842: Against the Rope

In [123]:
populate_stuff(movie_id_list[100:200], failed_id_list)

Trying movie no.0 with id 14846: Little Black Book (2004)
Trying movie no.1 with id 83: Open Water (2004)
Trying movie no.2 with id 11130: The Princess Diaries 2: Royal Engagement (2004)
Trying movie no.3 with id 395: Alien vs. Predator (2004)
Trying movie no.4 with id 11052: Yu-Gi-Oh! The Movie: Pyramid of Light (2004)
Trying movie no.5 with id 11026: Exorcist: The Beginning (2004)
Trying movie no.6 with id 10762: Without a Paddle (2004)
Trying movie no.7 with id 11237: Anacondas: The Hunt for the Blood Orchid (2004)
Trying movie no.8 with id 4836: Layer Cake (2004)
Trying movie no.9 with id 8080: Suspect Zero (2004)
Trying movie no.10 with id 4974: 5x2 (2004)
Trying movie no.11 with id 11632: Vanity Fair (2004)
Trying movie no.12 with id 18475: The Cookout (2004)
Trying movie no.13 with id 15644: Paparazzi (2004)
Trying movie no.14 with id 11208: Wicker Park (2004)
Trying movie no.15 with id 9759: Cellular (2004)
Trying movie no.16 with id 14198: Criminal (2004)
Trying movie no.17 wi